### `RandomForest` Modelling on Hyperglycemic data with imbalanced labelling

In [1]:
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
SEED=90089

In [2]:
# Read Data
patient_df = pd.read_csv('../data/hyperglycemic_patients_w_categories.csv', index_col=0)

# print detail summary
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (14077, 19)
Dataframe Features: Index(['subject_id', 'stay_id', 'glucose_max', 'anchor_age', 'dod', 'gender',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'apsiii', 'glucose_score', 'los',
       'avg_bmi_value', 'log10_los', 'label'],
      dtype='object')


,subject_id,stay_id,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los,avg_bmi_value,log10_los,label
0,12041046,31249096,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,92.000000,17.346154,37.186250,38,3,1.938414,26.750000,0.287447,0
1,17421995,38100564,2340.0,35,0,0,77.769231,122.961538,468.592593,105.361111,96.093023,19.813953,36.727586,87,5,4.472905,30.558477,0.650590,0
2,10352416,35043893,531.0,23,1,0,64.169118,119.661765,275.352941,154.028986,83.698413,25.584615,39.725000,158,5,11.636042,30.558477,1.065805,1
3,16924291,34261137,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,70.000000,26.333333,31.900000,88,3,0.309664,30.558477,-0.509109,2
4,15705944,37056020,406.0,24,0,0,57.333333,92.848485,292.000000,86.631579,96.162162,15.289474,33.590000,150,5,17.513646,30.558477,1.243377,1


In [3]:
print("Null value in Feature set summary:\n",patient_df.isnull().sum(), "\n----------------")
print("Label Distirbution:\n", patient_df['label'].value_counts())

Null value in Feature set summary:
 subject_id          0
stay_id             0
glucose_max         0
anchor_age          0
dod                 0
gender              0
dbp_mean            0
sbp_mean            0
glucose_mean        0
heart_rate_mean     0
spo2_mean           0
resp_rate_mean      0
temperature_mean    0
apsiii              0
glucose_score       0
los                 0
avg_bmi_value       0
log10_los           0
label               0
dtype: int64 
----------------
Label Distirbution:
 0    6461
2    4316
1    3300
Name: label, dtype: int64


In [163]:
patient_df['label'].value_counts()

0    6461
2    4316
1    3300
Name: label, dtype: int64

In [4]:
numerical_features = ['glucose_max', 'anchor_age', 'dbp_mean', 
                    'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean', 
                    'resp_rate_mean', 'temperature_mean', 'apsiii', 'glucose_score', 'avg_bmi_value']

# Standardized variable for consistent measurement across numerical values
scaler = StandardScaler()
X_std = scaler.fit_transform(patient_df[numerical_features])

# Interpolate categorical variables
X_cat = patient_df[['dod', 'gender']]


X = np.concatenate([X_std, X_cat.to_numpy()], axis=1)
y = patient_df["los"]
print("X Standardized data Shape: ", X_std.shape)
print("X Categorical data shape:  ", X_cat.shape)
print("X, y shape:                ", X.shape, y.shape)

X Standardized data Shape:  (14077, 12)
X Categorical data shape:   (14077, 2)
X, y shape:                 (14077, 14) (14077,)


In [5]:
# Split data based on training 
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=SEED)
print("Train data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Train data shape: (11261, 14) (11261,)
Test data shape: (2816, 14) (2816,)


### Hyperparameter Tunning on `RandomForest`

In [6]:
param_grid = {
 'bootstrap'        : [True, False],
 'max_depth'        : [20, 40, 60, 80, 100, None],
 'min_samples_leaf' : [1, 4],
 'max_features'     : ['sqrt', 'log2'],
 'min_samples_split': [2, 5, 10],
 'n_estimators'     : [600, 1000, 1400]}
rf_model = RandomForestRegressor(random_state=SEED)

In [7]:
# disable deprecation warnings
warnings.filterwarnings('ignore')

In [8]:
random_grid_rf = RandomizedSearchCV(rf_model, param_distributions=param_grid, 
                                    n_iter=100, cv=5, n_jobs=6)
random_grid_rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_RF.best_estimator_)

## Removed scoring due to combinations yielding NaN results
#  print("\n The best score across ALL searched params:\n", grid_RF.best_score_) 
print("\n The best parameters across ALL searched params:\n", grid_RF.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 RandomForestClassifier(max_depth=20, n_estimators=600, random_state=90089)

 The best parameters across ALL searched params:
 {'bootstrap': True, 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 600}


In [9]:
# Instantiate SVC classifierr
rf_clf = RandomForestRegressor(n_estimators=600, 
                                min_samples_split=2,
                                min_samples_leaf=1,
                                max_features='auto',
                                max_depth=20,
                                random_state=SEED,
                                n_jobs=4)

### Evaluation on `RandomForest` with imbalance data

In [10]:
rf_clf.fit(X_train, y_train)
print("Model Accuracy based on Testing dataset:", rf_clf.score(X_test, y_test))

Model Accuracy based on Testing dataset: 0.17669019183350232


In [158]:
y_pred = rf_clf.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.73      0.53      0.62      1777
           1       0.43      0.57      0.49       508
           2       0.36      0.57      0.44       531

    accuracy                           0.55      2816
   macro avg       0.51      0.56      0.52      2816
weighted avg       0.60      0.55      0.56      2816

